In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path

pd.set_option("display.max_rows", 100)
pd.set_option("display.min_rows", 100)
pd.set_option("display.max_columns", 100)


def make_sim_applecations_csv(
        start="2024-01-01",
        end="2024-12-31",
        base=50,
        noise_std=8,
        seed=42,
        out_path=Path("csv/sim_applications.csv"),
):
    """
    日別の申込件数(applications)を月効果、曜日効果で増減させて作る。
    - base: 平均の土台
    - noise_std: ばらつき具合
    """

    rng =  np.random.default_rng(seed)

    # 1) 日付レンジ
    df = pd.DataFrame({"date": pd.date_range(start=start, end=end, freq="D")})
    df["date"] = pd.to_datetime(df["date"])

    # 2 ) 月と曜日を特徴量として作る
    df["month"] = df["date"].dt.month # 1~12月
    df["weekday"] = df["date"].dt.weekday # 0(月) ~ 6(日)

    # 3 ) 月ごとの増減 例: 新生活、年度末(3)、 年末年始セール(12,1)は増える、夏(8)は落ちる等
    month_effect = {
        1: +12,  2: +30,  3: +50, 4: +10,
        5: +2,  6:  -30,  7: -20, 8: -13,
        9: +3, 10: +12, 11: +6, 12: +18
    }

    # 4) 曜日ごとの増減（例：土日は増える、平日は普通、など）
    weekday_effect = {
        0:  3,   # 月
        1:  0,   # 火
        2:  0,   # 水
        3:  0,   # 木
        4:  2,   # 金
        5:  15,   # 土
        6:  20   # 日
    }

    # 5) 効果を列に足し合わせる
    df["month_effect"] = df["month"].map(month_effect)
    df["weekday_effect"] = df["weekday"].map(weekday_effect)

    noise = rng.normal(0, noise_std, size=len(df))

    df["applications"] = (base + df["month_effect"] + df["weekday_effect"] + noise).round().astype(int)
    
    # 0未満が出ないように最低0に収める
    df["applications"] = df["applications"].clip(lower=0)

    # 保存
    out_df = df[["date", "applications", "month", "weekday"]]
    out_df.to_csv(out_path, index=False, encoding="utf-8")
    
    return out_df

if __name__ == "__main__":
    df = make_sim_applecations_csv()
    print(df.head(3))
    print(df.tail(3))
    print("saved:", "sim_applications.csv")


        date  applications  month  weekday
0 2024-01-01            67      1        0
1 2024-01-02            54      1        1
2 2024-01-03            68      1        2
          date  applications  month  weekday
363 2024-12-29            80     12        6
364 2024-12-30            70     12        0
365 2024-12-31            80     12        1
saved: sim_applications.csv

